In [1]:
# Scientific libraries		              
import numpy as np		              
import scipy as sp		              
import pandas as pd		              
import json		              
import missingno as msno		              

# Loading Plotting Modules		              
import matplotlib		              
import matplotlib.pyplot as plt		              
import seaborn as sns		              
%matplotlib inline		              
import chart_studio.plotly as py		              
import plotly.figure_factory as ff		              
import plotly.graph_objects as go		              
import plotly.express as px		              
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot		              
init_notebook_mode(connected=True)		              
		              
import os		              
import gc

In [2]:
# Setting Data Frame Options                      
pd.set_option('display.max_rows', 40)                      
pd.set_option('display.max_columns', 50)                      
pd.set_option('display.width', 100)                      
pd.set_option('display.max_colwidth', 100)                      

# Setting Plot Configuration                      
sns.set(rc={'figure.figsize':(19,11)}, style = 'white')

In [3]:
test = pd.read_csv('../../../data/test.csv')          
# test = pd.read_csv('../../eda/outputs/imputed_test_mf.csv')          
m, n = test.shape
print('Data Frame: {} x {}'.format(m,n))                
test.head()

Data Frame: 890 x 79


,galactic year,galaxy,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,"Intergalactic Development Index (IDI), Rank",Population using at least basic drinking-water services (%),Population using at least basic sanitation services (%),Gross capital formation (% of GGP),"Population, total (millions)","Population, urban (%)","Mortality rate, under-five (per 1,000 live births)","Mortality rate, infant (per 1,000 live births)",Old age dependency ratio (old age (65 and older) per 100 creatures (ages 15-64)),"Population, ages 15–64 (millions)","Population, ages 65 and older (millions)","Life expectancy at birth, male (galactic years)","Life expectancy at birth, female (galactic years)","Population, under age 5 (millions)",Young age (0-14) dependency ratio (per 100 creatures ages 15-64),...,Renewable energy consumption (% of total final energy consumption),"Estimated gross galactic income per capita, male","Estimated gross galactic income per capita, female",Rural population with access to electricity (%),Domestic credit provided by financial sector (% of GGP),"Population with at least some secondary education, female (% ages 25 and older)","Population with at least some secondary education, male (% ages 25 and older)",Gross fixed capital formation (% of GGP),"Remittances, inflows (% of GGP)",Population with at least some secondary education (% ages 25 and older),Intergalactic inbound tourists (thousands),"Gross enrolment ratio, primary (% of primary under-age population)","Respiratory disease incidence (per 100,000 people)",Interstellar phone subscriptions (per 100 people),"Interstellar Data Net users, total (% of population)",Current health expenditure (% of GGP),"Intergalactic Development Index (IDI), female","Intergalactic Development Index (IDI), male",Gender Development Index (GDI),"Intergalactic Development Index (IDI), female, Rank","Intergalactic Development Index (IDI), male, Rank",Adjusted net savings,"Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total",Private galaxy capital flows (% of GGP),Gender Inequality Index (GII)
0,1007012,KK98 77,0.456086,51.562543,12236.576447,0.593325,10.414164,10.699072,0.547114,0.556267,232.621842,105.193088,64.241392,17.418350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1007012,Reticulum III,0.529835,57.228262,3431.883825,0.675407,7.239485,5.311122,0.497688,0.409969,247.580771,55.730638,46.217440,33.027886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1008016,Reticulum III,0.560976,59.379539,27562.914252,0.594624,11.774890,5.937797,0.544744,0.486167,249.798771,58.755900,48.897493,31.613362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1007012,Segue 1,0.565910,59.952390,20352.232905,0.837700,11.613621,10.067882,0.691641,0.523441,211.505060,70.176431,61.254104,34.399007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1013042,Virgo I,0.588274,55.428320,23959.704016,0.520579,10.392416,6.374637,0.530676,0.580418,234.721069,69.768692,51.316940,15.657091,1106.554194,64.382217,219.588961,129.974418,14.013421,474.877713,21.305665,59.144562,63.990109,80.409057,111.084652,...,96.028122,21309.50697,18103.974021,50.233592,86.296925,29.065459,58.562225,13.313253,NaN,42.980727,99276.25673,NaN,458.186555,79.177012,41.585873,7.357729,0.583373,0.600445,0.856158,206.674424,224.104054,NaN,7.687626,NaN,NaN


    Potential for increase in the Index = -np.log(Index+0.01)+3

In [4]:
# RMSE Train: 0.001800391617117943
# RMSE Valid: 0.052504038023260875

# RMSE Train:  0.00438560
# RMSE Valid:  0.01734807	Ratio (val/train):  3.95569

preds = pd.read_csv('../outputs/y_pred_lgbm_mf_01317val.csv')
preds['opt_pred'] = 0
preds['eei'] = test['existence expectancy index']
preds['p_increase'] = -np.log(preds['y_pred'] + 0.01) + 3
preds['p_increase2'] = preds['p_increase'] ** 2
preds

,Index,y_pred,opt_pred,eei,p_increase,p_increase2
0,0,0.043461,0,0.456086,5.928805,35.150729
1,1,0.046403,0,0.529835,5.875224,34.518258
2,2,0.111482,0,0.560976,5.107988,26.091538
3,3,0.173968,0,0.565910,4.692996,22.024208
4,4,0.213073,0,0.588274,4.500258,20.252325
...,...,...,...,...,...,...
885,885,0.041134,0,NaN,5.973311,35.680446
886,886,0.048841,0,NaN,5.832922,34.022973
887,887,0.065078,0,NaN,5.589226,31.239449
888,888,0.040764,0,NaN,5.980577,35.767306


### Optimization

We will try to maximize sum of likely increase index for all galaxies, which for one galaxy is described as follows:

    Likely increase in the Index = extra energy * Potential for increase in the Index ** 2 / 1000

with respect to the following constraints.

* in total there are 50000 zillion DSML available for allocation
* no galaxy should be allocated more than 100 zillion DSML or less than 0 zillion DSML
* galaxies with low existence expectancy index below 0.7 should be allocated at least 10% of the total energy available

In [5]:
from scipy.optimize import minimize

In [6]:
preds['is_low'] = False
preds.loc[preds.eei < 0.7, 'is_low'] = True
preds.sort_values(by='is_low', ascending=False, inplace=True)
preds.reset_index(drop=True, inplace=True)
preds

,Index,y_pred,opt_pred,eei,p_increase,p_increase2,is_low
0,0,0.043461,0,0.456086,5.928805,35.150729,True
1,50,0.051282,0,0.676239,5.792277,33.550476,True
2,36,0.045148,0,0.652121,5.897741,34.783348,True
3,37,0.080642,0,0.656685,5.400843,29.169105,True
4,38,0.043686,0,0.656738,5.924594,35.100816,True
...,...,...,...,...,...,...,...
885,342,0.070584,0,0.903199,5.518454,30.453331,False
886,343,0.109042,0,0.903945,5.128278,26.299232,False
887,344,0.198459,0,0.903964,4.568014,20.866751,False
888,345,0.039338,0,0.904205,6.009066,36.108871,False


In [7]:
def objective(energy, p2):
    return -sum((energy * p2) / 1000)

In [8]:
%%time
m = test.shape[0]
bounds = [(0,100) for _ in range(m)]

# low_p2 = preds.loc[preds['eei'] < 0.7,'p_increase2'].copy().tolist()
# high_p2 = preds.loc[preds['eei'] >= 0.7,'p_increase2'].copy().tolist()
m_low = preds.loc[preds['eei'] < 0.7,'p_increase2'].shape[0]
p2 = preds['p_increase2']

cons = (
    {"type": "ineq", "fun": lambda x: 49999.999 - sum(x)},
    {"type": "ineq", "fun": lambda x: sum(x[:m_low]) - 5000.001}
)

history = minimize(objective, x0=np.zeros(m),
                   args=(p2),
                   bounds=bounds, constraints=cons)

Wall time: 1min 58s


In [9]:
preds['opt_pred'] = history.x
preds

,Index,y_pred,opt_pred,eei,p_increase,p_increase2,is_low
0,0,0.043461,9.869181e+01,0.456086,5.928805,35.150729,True
1,50,0.051282,9.768053e+01,0.676239,5.792277,33.550476,True
2,36,0.045148,9.966859e+01,0.652121,5.897741,34.783348,True
3,37,0.080642,9.238237e+01,0.656685,5.400843,29.169105,True
4,38,0.043686,1.000000e+02,0.656738,5.924594,35.100816,True
...,...,...,...,...,...,...,...
885,342,0.070584,5.032962e+01,0.903199,5.518454,30.453331,False
886,343,0.109042,2.174256e+01,0.903945,5.128278,26.299232,False
887,344,0.198459,4.201907e-11,0.903964,4.568014,20.866751,False
888,345,0.039338,8.929349e+01,0.904205,6.009066,36.108871,False


In [10]:
print('Total maximum likely increase:', -objective(history.x, p2))
print('Energy allocated (total):', preds.opt_pred.sum())
print('Energy allocated (low):', preds[preds.is_low].opt_pred.sum())

Total maximum likely increase: 1683.0812275639244
Energy allocated (total): 49999.998999690564
Energy allocated (low): 5000.00099999981


In [12]:
submission_file = preds[['Index', 'y_pred', 'opt_pred']]
submission_file.rename(columns={
    "Index": "index",
    "y_pred": "pred",
}, inplace=True)
submission_file.to_csv('../../../submissions/submitted/submission_5_lgb_01317.csv', index=False)